# Model Training

In this tutorial we are going to train a model from scratch on a molecular dataset from the MD17 collection.
Start by creating a project folder and downloading the dataset.

## Acquiring a dataset

You can obtain the benzene dataset with DFT labels either by running the following command or manually from this [link](http://www.quantum-machine.org/gdml/data/xyz/benzene2018_dft.zip). Apax uses ASE to read in datasets, so make sure to convert your own data into an ASE readable format (extxyz, traj etc). Be carefull the downloaded dataset has to be modified like in the `apax.untils.dataset.mop_md17` function in order to be readable.

In [1]:
from pathlib import Path
from apax.utils.datasets import download_md17_benzene_DFT, mod_md17

data_path = Path("project")

file_path = download_md17_benzene_DFT(data_path)
file_path = mod_md17(file_path)

## Configuration files

Next, we require a configuration file that specifies the model and training parameters.
In order to get users quickly up and running, our command line interface provides an easy way to generate input templates.
The provided templates come in in two levels of verbosity: minimal and full.
In the following we are going to use a minimal input file. To see a complete list and explanation of all parameters, consult the documentation page LINK.
For more information on the CLI,  simply run `apax -h`.

In [ ]:
!apax -h

The following command create a minimal configuration file in the working directory.

In [ ]:
!apax template train

Open the resulting `config.yaml` file in an editor of your choice and make sure to fill in the data path field with the name of the data set you just downloaded.
For the purposes of this tutorial we will train on 1000 data points and validate the model on 200 more during the training. Further, the units of the labels have to be specified. Random splitting is done by apax but it is also possible to input a pre-splitted training and validation dataset

The filled in configuration file should look similar to this one.

```yaml
epoch: 1000
data:
    data_path: md17.extexyz
    epochs: 1000
    n_train: 1000
    energy_unit: kcal/mol
    pos_unit: Ang
    ....
```

It also can be modefied with the utils function `mod_config` provided by Apax.


In [ ]:
from apax.utils.helpers import mod_config
import yaml


config_path = Path("config.yaml")

config_updates = {
    "n_epochs": 10,
    "data": {
        "experiment": "benzene_dft_cli",
        "directory": "project/models",
        "data_path": str(file_path),
        "energy_unit": "kcal/mol",
        "pos_unit": "Ang",
    }
}
config_dict = mod_config(config_path, config_updates)

with open("config.yaml", "w") as conf:
    yaml.dump(config_dict, conf, default_flow_style=False)


In order to check whether the a configuration file is valid, we provide the `validate` command. This is especially convenient when submitting training runs on a compute cluster.


In [ ]:
!apax validate train config.yaml

Configuration files are validated using Pydantic and the errors provided by the `validate` command give precise instructions on how to fix the input file.
For example, changing `epochs` to `-1000`, validate will give the following feedback to the user:

In [ ]:
config_updates = {
    "n_epochs": -1000,
}
config_dict = mod_config(config_path, config_updates)

with open("error_config.yaml", "w") as conf:
    yaml.dump(config_dict, conf, default_flow_style=False)

In [ ]:
!apax validate train error_config.yaml

## Training

Model training can be started by running

In [ ]:
!apax train config.yaml



During training, apax displays a progress bar to keep track of the validation loss.
This progress bar is optional however and can be turned off in the config. LINK
The default configuration writes training metrics to a CSV file, but TensorBoard is also supported.
One can specify which to use by adding the following section to the input file:

```yaml
callbacks:
    - CSV
```

If training is interrupted for any reason, re-running the above `train` command will resume training from the latest checkpoint.

Furthermore, an Apax trianing can easily be started within a script.

In [ ]:
from apax.train.run import run

config_path = Path("config.yaml")

config_updates = {
    "n_epochs": 100,
    "data": {
        "experiment": "benzene_dft_script",
        "directory": "project/models",
        "data_path": str(file_path),
        "energy_unit": "kcal/mol",
        "pos_unit": "Ang",
    }
}

config_dict = mod_config(config_path, config_updates)

run(config_dict)

In [ ]:
import csv
import matplotlib.pyplot as plt
import numpy as np


path = "project/models/benzene_dft_script/log.csv"

keys = ["energy_mae", "forces_mse", "forces_mae", "loss"]
data_dict = {}

with open(path, 'r') as file:
    reader = csv.reader(file)

    # Extract the headers (keys) from the first row
    headers = next(reader)

    # Initialize empty lists for each key
    for header in headers:
        data_dict[header] = []

    # Read the rest of the rows and append values to the corresponding key
    for row in reader:
        for idx, value in enumerate(row):
            key = headers[idx]
            data_dict[key].append(float(value))

fig, axes = plt.subplots(2, 2, constrained_layout=True)
axes = axes.ravel()
fig.suptitle(f'Metrics', fontsize=16)

for id, key in enumerate(keys):
    val = np.array(data_dict[f"val_{key}"])
    train = np.array(data_dict[f"train_{key}"])
    epoch = np.array(data_dict["epoch"])

    axes[id].plot(epoch, val, label="val data")
    axes[id].plot(epoch, train, label="train data")

    axes[id].set_ylabel(f"{key}")
    axes[id].set_xlabel(r"epoch")

plt.show()

## Evaluation

After the training is completed and we are satisfied with our choice of hyperparameters and vadliation loss, we can evaluate the model on the test set.
We provide a separate command for test set evaluation:


In [ ]:
from apax.train.eval import eval_model

eval_model(config_dict, n_test=100)
# !apax eval config.yaml


TODO pretty print results to the terminal

Congratulations, you have successfully trained and evaluated your first apax model!

## A Closer Look At Training Parameters

```yaml
n_epochs: <NUMBER OF EPOCHS> # Number of training epochs.
seed: 1 # Seed for initialising random numbers
patience: None # Number of epochs without improvement before trainings gets terminated.
n_models: 1 # Number of models to be trained at once.
n_jitted_steps: 1 # Number of train batches to be processed in a compiled loop. Can yield singificant speedups for small structures or small batch sizes.

data:
  directory: models/ # Path to the directory where the training results and checkpoints will be written.
  experiment: apax # Name of  the model. Distinguishes it from the other models trained in the same `directory`.
  data_path: <PATH> # Path to a single dataset file. Set either this or `val_data_path` and `train_data_path`.
  train_data_path: <PATH> # Path to a training dataset. Set this and `val_data_path` if your data comes pre-split.
  val_data_path: <PATH> # Path to a validation dataset. Set this and `train_data_path` if your data comes pre-split.
  test_data_path: <PATH> # Path to a test dataset. Set this, `train_data_path` and `val_data_path` if your data comes pre-split.

  n_train: 1000 # Number of training datapoints from `data_path`.
  n_valid: 100 # Number of validation datapoints from `data_path`.

  batch_size: 32 # Number of training examples to be evaluated at once.
  valid_batch_size: 100 # Number of validation examples to be evaluated at once.

  shift_method: "per_element_regression_shift"
  shift_options:
    energy_regularisation: 1.0 # Magnitude of the regularization in the per-element energy regression.
  shuffle_buffer_size: 1000 # Size of the `tf.data` shuffle buffer.

  pos_unit: Ang
  energy_unit: eV

  additional_properties_info: # Dict of property name, shape (ragged or fixed) pairs

model:
  n_basis: 7 # Number of uncontracted gaussian basis functions.
  n_radial: 5 # Number of contracted basis functions.
  nn: [512, 512] # Number of hidden layers and units in those layers.

  r_max: 6.0 # Position of the first uncontracted basis function's mean.
  r_min: 0.5 # Cutoff radius of the descriptor.

  use_zbl: false # 

  b_init: normal # Initialization scheme for the neural network biases. Either `normal` or `zeros`.
  descriptor_dtype: fp64
  readout_dtype: fp32
  scale_shift_dtype: fp32

loss:
- loss_type: structures # Weighting scheme for atomic contributions. See the MLIP package for reference 10.1088/2632-2153/abc9fe for details
  name: energy # Keyword of the quantity e.g `energy`.
  weight: 1.0 # Weighting factor in the overall loss function.
- loss_type: structures
  name: forces
  weight: 4.0

metrics:
- name: energy # Keyword of the quantity e.g `energy`.
  reductions: # List of reductions performed on the difference between target and predictions. Can be mae, mse, rmse for energies and forces. For forces it is also possible to use `angle`.
  - mae
- name: forces
  reductions:
  - mae
  - mse

optimizer:
  opt_name: adam # Name of the optimizer. Can be any `optax` optimizer.
  opt_kwargs: {} # Optimizer keyword arguments. Passed to the `optax` optimizer.
  emb_lr: 0.03 # Learning rate of the elemental embedding contraction coefficients.
  nn_lr: 0.03 # Learning rate of the neural network parameters.
  scale_lr: 0.001 # Learning rate of the elemental output scaling factors.
  shift_lr: 0.05 # Learning rate of the elemental output shifts.
  zbl_lr: 0.001 # 
  transition_begin: 0 # Number of training steps (not epochs) before the start of the linear learning rate schedule.

callbacks:
- name: csv # Keyword of the callback used. Currently we implement "csv" and "tensorboard".

progress_bar:
  disable_epoch_pbar: false # Set to True to disable the epoch progress bar.
  disable_nl_pbar: false # Set to True to disable the NL precomputation progress bar.


checkpoints:
  ckpt_interval: 1 # Number of epochs between checkpoints.
  # The options below are used for transfer learning
  base_model_checkpoint: null # Path to the folder containing a pre-trained model ckpt.
  reset_layers: [] # List of layer names for which the parameters will be reinitialized.

```

To remove all the created files and clean up yor working directory run

In [ ]:
# !rm -r project config.yaml error_config.yaml